## Network Analysis
### 1.1 Centrality Measures
Select 3 centrality measures to characterise nodes, aiming at identifying the most important nodes in 
the underground network. Give the definition of each of the measures (including their equation), put 
the measures into the context of the underground, and why they will allow you to find the stations that 
are most crucial for the functioning of the underground. Compute the measures for your nodes in the 
network, and give the results in a table for the first 10 ranked nodes for each of the 3 measures.  

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
from operator import itemgetter

1.1.1 Read the data, select centrality measures:


In [2]:
london_graph = nx.read_graphml('london_tubenetwork.graphml')

FileNotFoundError: [Errno 2] No such file or directory: 'london_tubenetwork.graphml'

In [4]:
print(len(dir(london_graph)))
for i in np.random.randint(0, len(dir(london_graph)), 10):
    print(dir(london_graph)[i])

75
graph
__doc__
edge_subgraph
__subclasshook__
__ge__
add_node
__dir__
_node
remove_edge
__getitem__


In [5]:
print(london_graph.number_of_nodes())
print(london_graph.number_of_edges())

438
486


In [7]:
# Recompute the centrality measures
degree_centrality = nx.degree_centrality(london_graph)
betweenness_centrality = nx.betweenness_centrality(london_graph)
closeness_centrality = nx.closeness_centrality(london_graph)

# Sort and get the top 10 nodes for each centrality measure
sorted_degree = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
sorted_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

(sorted_degree, sorted_betweenness, sorted_closeness)


([('940GZZLUKSX', 0.016018306636155607),
  ('940GZZLUBST', 0.016018306636155607),
  ('940GZZLUGPK', 0.013729977116704806),
  ('940GZZLUOXC', 0.013729977116704806),
  ('940GZZLUECT', 0.013729977116704806),
  ('940GZZLUBNK', 0.013729977116704806),
  ('940GZZLUWLO', 0.013729977116704806),
  ('940GZZLUTNG', 0.011441647597254004),
  ('940GZZLULVT', 0.011441647597254004),
  ('940GZZLUWHM', 0.011441647597254004)],
 [('940GZZLUBST', 0.3810150084358616),
  ('940GZZLUBLG', 0.3534325817535458),
  ('940GZZLUFYR', 0.3365817857034552),
  ('940GZZLUBNK', 0.3195625056858345),
  ('940GZZLUGPK', 0.31955197127241747),
  ('940GZZLUWLO', 0.3172160057103268),
  ('940GZZLULVT', 0.3130260708612372),
  ('940GZZLUWSM', 0.2899622285670293),
  ('940GZZLUBND', 0.2585985889467725),
  ('910GWHMDSTD', 0.23656559877955732)],
 [('940GZZLUGPK', 0.09489685124864278),
  ('940GZZLUBND', 0.09373659373659374),
  ('940GZZLUWSM', 0.09319684367669012),
  ('940GZZLUBST', 0.09289965986394558),
  ('940GZZLUWLO', 0.0923890063424947

In [ ]:
# Compute initial betweenness centrality
initial_betweenness = nx.betweenness_centrality(G)

# Initialize a dictionary to store delta centrality
delta_centrality = {}

# Iterate over all nodes to compute delta centrality
for node in G.nodes():
    # Create a copy of the graph without the current node
    G_copy = copy.deepcopy(G)
    G_copy.remove_node(node)
    
    # Recompute betweenness centrality without the node
    new_betweenness = nx.betweenness_centrality(G_copy)
    
    # Calculate the difference and store it
    delta_centrality[node] = sum(abs(initial_betweenness[n] - new_betweenness.get(n, 0)) for n in initial_betweenness)

# Sort nodes by their delta centrality
sorted_delta_centrality = sorted(delta_centrality.items(), key=lambda x: x[1], reverse=True)

Delta centrality

In [ ]:
import networkx as nx
import copy

# Load your network
G = nx.read_graphml('path_to_your_graphml_file.graphml')

# Compute initial betweenness centrality
initial_betweenness = nx.betweenness_centrality(G)

# Initialize a dictionary to store delta centrality
delta_centrality = {}

# Iterate over all nodes to compute delta centrality
for node in G.nodes():
    # Create a copy of the graph without the current node
    G_copy = copy.deepcopy(G)
    G_copy.remove_node(node)
    
    # Recompute betweenness centrality without the node
    new_betweenness = nx.betweenness_centrality(G_copy)
    
    # Calculate the difference and store it
    delta_centrality[node] = sum(abs(initial_betweenness[n] - new_betweenness.get(n, 0)) for n in initial_betweenness)

# Sort nodes by their delta centrality
sorted_delta_centrality = sorted(delta_centrality.items(), key=lambda x: x[1], reverse=True)


Find 2 different measures to evaluate the impact of the node removal on the network. These need to 
be global measures referring to the whole network and not to specific nodes or links. Explain whether 
these two measures are specific to the London underground, or whether they could also be used to 
evaluate the resilience of any other network.  

For each of the centrality measures selected in I.1. remove at least 10 nodes following two different 
strategies. A) Non-sequential removal: using the table created in I.1. remove 1 node at a time 
following the rank in the table, i.e. from the most important one to the 10

th most important one. After 
each removal, evaluate the impact of the removal using your two measures in I.2. and proceed until 
you have removed at least 10 nodes. B) Sequential: remove the highest ranked node and evaluate the 
impact using the 2 measures. After removal, re-compute the centrality measure. Remove the highest 
ranked node in the new network and evaluate the impact. Continue until removing at least 10 nodes. 

Report the results of the 2 strategies in one plot, and critically discuss the following: which centrality 
measure reflects better the importance of a station for the functioning of the underground, which 
strategy is more effective at studying resilience, and which impact measure is better at assessing the 
damage after node removal.  